In [43]:
#==================================================================
#                    INSTALLING DEPENDENCIES
#==================================================================

# import tensorflow as tf
# import glob
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# import librosa as lb
# import librosa.display
# import IPython.display as ipd
# import os
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D,Dense, Dropout,  MaxPooling2D, Flatten
# from tensorflow.keras.metrics import Accuracy, Precision
# from tensorflow.keras.utils import to_categorical
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Dropout, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import Accuracy, Precision

import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import librosa as lb
import librosa.display
import IPython.display as ipd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [19]:
#-------------------------------------------------------------------
#                       Feature Selection
#-------------------------------------------------------------------
# ipd.Audio(audio_files[0])


#using glob and librosa
audio_files = glob.glob("C:\\Datasets\\Data\\genres_original\\*\\*.wav")
SAMPLES_PER_TRACK = 22050 * 30 



def extract_mel_spectrogram(file_path, n_mels = 128):
    y, sr = librosa.load(file_path, duration=30)
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
    S_DB = librosa.power_to_db(S, ref=np.max)
    return S_DB



<>:8: SyntaxWarning: invalid escape sequence '\D'
<>:8: SyntaxWarning: invalid escape sequence '\D'
C:\Users\Administrator\AppData\Local\Temp\ipykernel_5232\1398052004.py:8: SyntaxWarning: invalid escape sequence '\D'
  audio_files = glob.glob("C:\Datasets\Data\genre_original")


In [49]:
#--------------------------------------------------------------------
#                        DATA PREPROCESSING
#--------------------------------------------------------------------

# data = []
# 
# for genre in genres:
#     folder = f"C:\Datasets\Data\genres_original\{genres}"
#     for filename in os.listdir(folder):
#         file_path = os.path.join(folder, filename)
#         features = extract_features(file_path)
#         data.append([*features, genre])

# df = pd.DataFrame(data)


#Below is just normal code for extracting features from DataSet doesnt really help much 
#Kept it as a memorial 
# data = []
# genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
# for genre in genres:
#     folder = f'C:\Datasets\Data\genres_original\{genre}'
#     for filename in os.listdir(folder):
#         file_path = os.path.join(folder, filename)
#         features = extract_features(file_path)
#         data.append([*features, genre])

# df = pd.DataFrame(data)




<string>:19: SyntaxWarning: invalid escape sequence '\{'
<>:19: SyntaxWarning: invalid escape sequence '\{'
<>:19: SyntaxWarning: invalid escape sequence '\D'
<string>:19: SyntaxWarning: invalid escape sequence '\{'
<>:19: SyntaxWarning: invalid escape sequence '\{'
<>:19: SyntaxWarning: invalid escape sequence '\D'
C:\Users\Administrator\AppData\Local\Temp\ipykernel_2332\1467794334.py:19: SyntaxWarning: invalid escape sequence '\{'
  folder = f'C:\Datasets\Data\genres_original\{genre}'
C:\Users\Administrator\AppData\Local\Temp\ipykernel_2332\1467794334.py:19: SyntaxWarning: invalid escape sequence '\D'
  folder = f'C:\Datasets\Data\genres_original\{genre}'


In [44]:
#loading Dataset ig
#Preparing for x:y key value thing
genres = "blues classical country disco hiphop jazz metal pop reggae rock".split(" ")
X, y = [], []
for genre in genres:
    folder = os.path.join(r"C:\Datasets\Data\genres_original", genre)
    for filename in os.listdir(folder):
        if filename.endswith(".wav"):
            filepath = os.path.join(folder, filename)
            mel = extract_mel_spectrogram(filepath)
            if mel.shape[1] >= 660:  # Ensure fixed width
                mel = mel[:, :660]   # Shape: (128, 660)
                X.append(mel)
                y.append(genre)

X = np.array(X)
X = X[..., np.newaxis]  # Shape: (samples, 128, 660, 1)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_cat = to_categorical(y_encoded)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_5232\1398052004.py:14: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, duration=30)


NoBackendError: 

In [30]:
print(genres)

['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']


In [65]:
#Previous Model (Sucks ass honestly) 
# Uses x:y system 
# music_model = Sequential([
#     Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
#     Dropout(0.3),
#     Dense(128, activation='relu'),
#     Dropout(0.3),
#     Dense(64, activation='relu'),
#     Dense(10, activation='softmax')
# ])

# music_model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy',metrics = ['accuracy', 'precision'] )

# music_model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test))

#Strings of feature 
x = df.iloc[:, :-1]  
y = df.iloc[:, -1]   

#This thing encodes above stuff
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Normal stuff yk (train test split) --> NO need to explain
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


#This Model specifically USES Spectrogram given in the data to train CNN(Conv2D and MaxPooling2D) to the music and trains itself
#Much better than previous stuff
music_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 660, 1)),
    MaxPooling2D((2, 2)),
    Dropout(0.3),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.3),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.3),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.4),
    Dense(10, activation='softmax')
])

music_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', 'precision'])

music_model.summary()
# Train
music_model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test))


Epoch 1/30


c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.0722 - loss: 2.3150 - val_accuracy: 0.0750 - val_loss: 2.3070
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0753 - loss: 2.3054 - val_accuracy: 0.0850 - val_loss: 2.3076
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1251 - loss: 2.3023 - val_accuracy: 0.0850 - val_loss: 2.3074
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0860 - loss: 2.3040 - val_accuracy: 0.1100 - val_loss: 2.3034
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0908 - loss: 2.3028 - val_accuracy: 0.0750 - val_loss: 2.3068
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1288 - loss: 2.3005 - val_accuracy: 0.0850 - val_loss: 2.3054
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1079 - loss: 2.3028 - val_accuracy: 0.1000 - val_loss: 2.3055
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0800 - loss: 2.3067 - val_accuracy: 0.1000 - val_loss: 2.3057
Ep

In [66]:
genre_names = label_encoder.classes_  # Use this to decode predictions


In [67]:
print(genre_names)

['blues' 'classical' 'country' 'disco' 'hiphop' 'jazz' 'metal' 'pop'
 'reggae' 'rock']
